### Conclusions
#### The universal machine-learning workflow
Having access to an extremely powerful tool for creating models that map any input space to any target space is great, but the difficult part of the machine learning workflow is often everything that comes before designing and training such models (and, for production models, what comes after, as well). Understanding the problem domain so as to be able to determine what to attempt to predict, given what data, and how to measure success, is a prerequisite for any successful application of machine learning, and it isn’t something that advanced tools like Keras and TensorFlow can help you with. As a reminder, here’s a quick summary of the typical machine learning workflow as described in chapter 6:
1. Define the problem: What data is available, and what are you trying to predict? Will you need to collect more data or hire people to manually label a dataset?
2. Identify a way to reliably measure success on your goal. For simple tasks this may be prediction accuracy, but in many cases it will require sophisticated, domain specific metrics.
3. Prepare the validation process that you’ll use to evaluate your models. In particular, you should define a training set, a validation set, and a test set. The validation- and test-set labels shouldn’t leak into the training data: for instance, with temporal prediction, the validation and test data should be posterior to the training data.
4. Vectorize the data by turning it into vectors and preprocessing it in a way that makes it more easily approachable by a neural network (normalization and so on).
5. Develop a first model that beats a trivial common-sense baseline, thus demonstrating that machine learning can work on your problem. This may not always be the case!
6. Gradually refine your model architecture by tuning hyperparameters and adding regularization. Make changes based on performance on the validation data only, not the test data or the training data. Remember that you should get your model to overfit (thus identifying a model capacity level that’s greater than you need) and only then begin to add regularization or downsize your model. Beware of validation-set overfitting when tuning hyperparameters—the fact that your hyperparameters may end up being overspecialized to the validation set. Avoiding this is the purpose of having a separate test set.
7. Deploy your final model in production—as a web API, as part of a JavaScript or C++ application, on an embedded device, etc. Keep monitoring its performance on real-world data, and use your findings to refine the next iteration of the model!

#### Key network architectures
The four families of network architectures that you should be familiar with are **densely connected networks, convolutional networks, recurrent networks**, and **Transformers**. Each type of model is meant for a specific input modality. A network architecture encodes assumptions about the structure of the data: a hypothesis space within which the search for a good model will proceed. Whether a given architecture will work on a given problem depends entirely on the match between the structure of the data and the assumptions of the network architecture. <br>
These different network types can easily be combined to achieve larger multimodal models, much as you combine LEGO bricks. In a way, deep learning layers are LEGO bricks for information processing. Here’s a quick overview of the mapping between input modalities and appropriate network architectures:
- **Vector data**—Densely connected models (Dense layers).
- **Image data**—2D convnets.
- **Sequence data**—RNNs for timeseries, or Transformers for discrete sequences (such as sequences of words). 1D convnets can also be used for translation invariant, continuous sequence data, such as birdsong waveforms.
- **Video data**—Either 3D convnets (if you need to capture motion effects), or a combination of a frame-level 2D convnet for feature extraction followed by a sequence-processing model.
- **Volumetric data**—3D convnets.

Now, let’s quickly review the specificities of each network architecture.

##### DENSELY CONNECTED NETWORKS
A densely connected network is a stack of **Dense** layers meant to process vector data (where each sample is a vector of numerical or categorical attributes). Such networks assume no specific structure in the input features: they’re called densely connected because the units of a Dense layer are connected to every other unit. The layer attempts to map relationships between any two input features; this is unlike a 2D convolution layer, for instance, which only looks at local relationships. <br>
Densely connected networks are most commonly used for categorical data (for example, where the input features are lists of attributes), such as the Boston Housing Price dataset used in chapter 4. They’re also used as the final classification or regression stage of most networks. For instance, the convnets covered in chapter 8 typically end with one or two Dense layers, and so do the recurrent networks in chapter 10. <br>
Remember, to perform **binary classification**, end your stack of layers with a **Dense** layer with a single unit and a **sigmoid** activation, and use **binary_crossentropy** as the loss. Your targets should be either 0 or 1:

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
inputs = keras.Input(shape=(num_input_features,))
x = layers.Dense(32, activation="relu")(inputs)
x = layers.Dense(32, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop", loss="binary_crossentropy")

To perform **single-label categorical classification** (where each sample has exactly one class, no more), end your stack of layers with a **Dense** layer with a number of units equal to the number of classes, and a **softmax** activation. If your targets are one-hot encoded, use **categorical_crossentropy** as the loss; if they’re integers, use **sparse_categorical_crossentropy**:

In [ ]:
inputs = keras.Input(shape=(num_input_features,))
x = layers.Dense(32, activation="relu")(inputs)
x = layers.Dense(32, activation="relu")(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")

To perform **multilabel categorical classification** (where each sample can have several classes), end your stack of layers with a **Dense** layer with a number of units equal to the number of classes, and a **sigmoid** activation, and use **binary_crossentropy** as the loss. Your targets should be multi-hot encoded:

In [ ]:
inputs = keras.Input(shape=(num_input_features,))
x = layers.Dense(32, activation="relu")(inputs)
x = layers.Dense(32, activation="relu")(x)
outputs = layers.Dense(num_classes, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop", loss="binary_crossentropy")

To perform **regression** toward a vector of continuous values, end your stack of layers with a **Dense** layer with a number of units equal to the number of values you’re trying to predict (often a single one, such as the price of a house), and **no activation**. Various losses can be used for regression—most commonly **mean_squared_error** (MSE):

In [ ]:
inputs = keras.Input(shape=(num_input_features,))
x = layers.Dense(32, activation="relu")(inputs)
x = layers.Dense(32, activation="relu")(x)
outputs = layers.Dense(num_values)(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop", loss="mse")

##### CONVNETS
Convolution layers look at spatially local patterns by applying the same geometric transformation to different spatial locations (patches) in an input tensor. This results in representations that are **translation invariant**, making convolution layers highly data efficient and modular. This idea is applicable to spaces of any dimensionality: 1D (continuous sequences), 2D (images), 3D (volumes), and so on. You can use the **Conv1D** layer to process sequences, the **Conv2D** layer to process images, and the **Conv3D** layers to process volumes. As a leaner, more efficient alternative to convolution layers, you can also use **depthwise separable convolution** layers, such as **SeparableConv2D**. <br>
**Convnets**, or **convolutional networks**, consist of stacks of convolution and max-pooling layers. The pooling layers let you spatially downsample the data, which is required to keep feature maps to a reasonable size as the number of features grows, and to allow subsequent convolution layers to “see” a greater spatial extent of the inputs. Convnets are often ended with either a **Flatten** operation or a global pooling layer, turning spatial feature maps into vectors, followed by **Dense** layers to achieve classification or regression. <br>
Here’s a typical image-classification network (categorical classification, in this case), leveraging **SeparableConv2D** layers:

In [ ]:
inputs = keras.Input(shape=(height, width, channels))
x = layers.SeparableConv2D(32, 3, activation="relu")(inputs)
x = layers.SeparableConv2D(64, 3, activation="relu")(x)
x = layers.MaxPooling2D(2)(x)
x = layers.SeparableConv2D(64, 3, activation="relu")(x)
x = layers.SeparableConv2D(128, 3, activation="relu")(x)
x = layers.MaxPooling2D(2)(x)
x = layers.SeparableConv2D(64, 3, activation="relu")(x)
x = layers.SeparableConv2D(128, 3, activation="relu")(x)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(32, activation="relu")(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")

When building a very deep convnet, it’s common to add **batch normalization** layers as well as **residual connections**—two architecture patterns that help gradient information flow smoothly through the network.

##### RNNS
**Recurrent neural networks** (RNNs) work by processing sequences of inputs one timestep at a time, and maintaining a state throughout (a state is typically a vector or set of vectors). They should be used preferentially over 1D convnets in the case of sequences where patterns of interest aren’t invariant by temporal translation (for instance, timeseries data where the recent past is more important than the distant past). <br>
Three RNN layers are available in Keras: **SimpleRNN, GRU,** and **LSTM**. For most practical purposes, you should use either **GRU** or **LSTM**. **LSTM** is the more powerful of the two but is also more expensive; you can think of **GRU** as a simpler, cheaper alternative to it. <br>
**In order to stack multiple RNN layers on top of each other, each layer prior to the last layer in the stack should return the full sequence of its outputs (each input timestep will correspond to an output timestep).** If you aren’t stacking any further RNN layers, it’s common to return only the last output, which contains information about the entire sequence. <br>
Following is a single RNN layer for **binary classification** of vector sequences:

In [ ]:
inputs = keras.Input(shape=(num_timesteps, num_features))
x = layers.LSTM(32)(inputs)
outputs = layers.Dense(num_classes, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop", loss="binary_crossentropy")

And this is a stacked RNN for binary classification of vector sequences:

In [ ]:
inputs = keras.Input(shape=(num_timesteps, num_features))
x = layers.LSTM(32, return_sequences=True)(inputs)
x = layers.LSTM(32, return_sequences=True)(x)
x = layers.LSTM(32)(x)
outputs = layers.Dense(num_classes, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop", loss="binary_crossentropy")

##### TRANSFORMERS
A Transformer looks at a set of vectors (such as word vectors), and leverages **neural attention** to transform each vector into a representation that is aware of the **context** provided by the other vectors in the set. When the set in question is an ordered sequence, you can also leverage **positional encoding** to create Transformers that can take into account both global context and word order, capable of processing long text paragraphs much more effectively than RNNs or 1D convnets. <br>
Transformers can be used for any set-processing or sequence-processing task, including text classification, but they excel especially at **sequence-to-sequence learning**, such as translating paragraphs in a source language into a target language. <br>
A **sequence-to-sequence Transformer** is made up of two parts:
- A **TransformerEncoder** that turns an input vector sequence into a context-aware, order-aware output vector sequence
- A **TransformerDecoder** that takes the output of the TransformerEncoder, as well as a target sequence, and predicts what should come next in the target sequence

If you’re only processing a single sequence (or set) of vectors, you’d be only using the **TransformerEncoder**. <br>
Following is a sequence-to-sequence Transformer for mapping a source sequence to a target sequence (this setup could be used for machine translation or question answering, for instance):

In [ ]:
encoder_inputs = keras.Input(shape=(sequence_length,), dtype="int64") # Source sequence
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
decoder_inputs = keras.Input(shape=(None,), dtype="int64") # Target sequence so far
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x) # Target sequence one step in the future
transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)
transformer.compile(optimizer="rmsprop", loss="categorical_crossentropy")

And this is a lone **TransformerEncoder** for binary classification of integer sequences:

In [ ]:
inputs = keras.Input(shape=(sequence_length,), dtype="int64")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop", loss="binary_crossentropy")

Full implementations of the TransformerEncoder, the TransformerDecoder, and the PositionalEmbedding layer are provided in chapter 11.

#### The space of possibilities
What will you build with these techniques? Remember, building deep learning models is like playing with LEGO bricks: layers can be plugged together to map essentially anything to anything, given that you have appropriate training data available and that the mapping is achievable via a continuous geometric transformation of reasonable complexity. The space of possibilities is infinite. This section offers a few examples to inspire you to think beyond the basic classification and regression tasks that have traditionally been the bread and butter of machine learning. <br>
I’ve sorted my suggested applications by input and output modalities in the following list. Note that quite a few of them stretch the limits of what is possible—although a model could be trained on all of these tasks, in some cases such a model probably wouldn’t generalize far from its training data.
- Mapping vector data to vector data:
  - **Predictive healthcare**—Mapping patient medical records to predictions of patient outcomes
  - **Behavioral targeting**—Mapping a set of website attributes with data on how long a user will spend on the website
  - **Product quality control**—Mapping a set of attributes relative to an instance of a manufactured product with the probability that the product will fail by next year
- Mapping image data to vector data:
  - **Medical assistant**—Mapping slides of medical images to a prediction about the presence of a tumor
  - **Self-driving vehicle**—Mapping car dashcam video frames to steering wheel angle commands and gas and braking commands
  - **Board game AI**—Mapping Go or chess boards to the next player move
  - **Diet helper**—Mapping pictures of a dish to its calorie count
  - **Age prediction**—Mapping selfies to the age of the person
- Mapping timeseries data to vector data:
  - **Weather prediction**—Mapping timeseries of weather data in a grid of locations to the temperature in a specific place one week later
  - **Brain-computer interfaces**—Mapping timeseries of magnetoencephalogram (MEG) data to computer commands
  - **Behavioral targeting**—Mapping timeseries of user interactions on a website to the probability that a user will buy something
- Mapping text to text:
  - **Machine translation**—Mapping a paragraph in one language to a translated version in a different language
  - **Smart reply**—Mapping emails to possible one-line replies
  - **Question answering**—Mapping general-knowledge questions to answers
  - **Summarization**—Mapping a long article to a short summary of the article
- Mapping images to text:
  - **Text transcription**—Mapping images that contain a text element to the corresponding text string
  - **Captioning**—Mapping images to short captions describing the contents of the images
- Mapping text to images:
  - **Conditioned image generation**—Mapping a short text description to images matching the description
  - **Logo generation/selection**—Mapping the name and description of a company to a logo suggestion
- Mapping images to images:
  - **Super-resolution**—Mapping downsized images to higher-resolution versions of the same images
  - **Visual depth sensing**—Mapping images of indoor environments to maps of depth predictions
- Mapping images and text to text:
  - **Visual QA**—Mapping images and natural language questions about the contents of images to natural language answers
- Mapping video and text to text:
  - **Video QA**—Mapping short videos and natural language questions about the contents of videos to natural language answers